# Preamble

In [1]:
%matplotlib notebook

## Notebook parameters

In [2]:
NAME = 'a_3_download_filing_deadlines'
PROJECT = 'covid-empirical'
PYTHON_VERSION = '3.9'
USER = 'Ties'
CONDA_ENVIRONMENT = 'covid-empirical'
USE_EXTERNAL_PIPELINE = True

## Run preamble script

In [3]:
%run -i preamble.py 

----------------------------------------------------------------------------------
The following utility functions are loaded and available through `functions.<..>`:
----------------------------------------------------------------------------------

extract_data_edgar_link, fast_load_json, fast_store_json, flatten_multiindex_column, inDB, recreate_edgar_link

----------------------------------------------------------------
The following modules and functions are imported by preamble.py:
----------------------------------------------------------------

copy, delayed, importlib, json, math, np, orjson, os, pd, plt, pqdm_p, pqdm_t, random, re, requests, sys, time, yaml


## Notebook specific imports

In [4]:
from datetime import datetime, date

----
<span class="cSpacer"></span>
# Generate a dataset with all the deadlines

## Requests logic

In [5]:
endpoint = "https://www.securexfilings.com/wp-content/plugins/deadlines-functions/ajax.php"

In [6]:
def get_deadline(filerType, dateItem, formType, endpoint=endpoint):
    payload = {
          "type" : filerType,
          "day" : dateItem.day,
          "month" : dateItem.month,
          "year" : dateItem.year,
          "calc_nonce_field":"7cca7b24fb", ## Retrieved through chrome devtools after using the website manually
          "form": formType
        }
    res = requests.get(endpoint, payload)
    res_text = res.text
    deadline_date = pd.to_datetime(re.findall('Due:(.*?)<', res_text)[0].strip())
    
    ft = '10-K' if formType == '10Kresult' else '10-Q'
    
    ret_dict = {
        'filerType' : filerType, 
        'periodEnd' : dateItem.strftime('%m-%d-%Y'),
        'formType' : ft,
        'deadline' : deadline_date.strftime('%m-%d-%Y')
    }
    
    return ret_dict

## Execute

In [7]:
type_list = ['naf', 'af', 'laf']
date_list = pd.date_range(start=date(2017, 1, 1),end=date(2021, 12, 31)).to_pydatetime().tolist()
form_list = ['10Kresult', '10Qresult']

In [ ]:
ret_list = []
issue_list = []
for filerType in type_list:
    for formType in form_list:
        for dateItem in tqdm(date_list):
            try:
                ret_list.append(get_deadline(filerType, dateItem, formType))
            except:
                print(filerType, dateItem, formType)
                issue_list.append((filerType, dateItem, formType))

In [63]:
pd.DataFrame(ret_list)

,filerType,periodEnd,formType,deadline
0,naf,01-01-2017,10-K,04-03-2017
1,naf,01-02-2017,10-K,04-03-2017
2,naf,01-03-2017,10-K,04-03-2017
3,naf,01-04-2017,10-K,04-04-2017
4,naf,01-05-2017,10-K,04-05-2017
...,...,...,...,...
1996,naf,06-20-2017,10-Q,08-04-2017
1997,naf,06-21-2017,10-Q,08-07-2017
1998,naf,06-22-2017,10-Q,08-07-2017
1999,naf,06-23-2017,10-Q,08-07-2017


### Convert to dataframe

In [69]:
ret_df = pd.DataFrame(ret_list)

## Store as Excel sheet

In [70]:
ret_df.to_excel(pipeline / 'out' / 'deadline_df.xlsx')